# **GATK Somatic Mutation Calling on Breast Cancer Cell Line HCC1143 (BAM → MAF).**

## Dataset 

In this analysis, we process WES data from the well-characterized breast cancer cell line HCC1143 and matched normal sample. This pair of samples is widely used in benchmarking and demonstration workflows and is supported by publicly available reference results, enabling comparison and validation of somatic variant calling outputs. 

To reduce computational complexity the analysis is restricted to **chromosome 17**, which harbors several clinically relevant genes (e.g. TP53).

## Data source 

Input data and reference resources were obtained from official Broad Institute repositories:

**Germline population resource (gnomAD, ExAC):**

https://console.cloud.google.com/storage/browser/gatk-best-practices/somatic-b38

Files used: af-only-gnomad.hg38.vcf.gz, af-only-gnomad.hg38.vcf.gz.tbi, small_exac_common_3.hg38.vcf.gz, small_exac_common_3.hg38.vcf.gz.tbi

**WES BAM files (tumor and matched normal):**

https://console.cloud.google.com/storage/browser/gatk-tutorials/workshop_1903/3-somatic

Files used: bams/tumor.bam,tumor.bai, normal.bam, normal.bai., refs/Homo_sapiens_assembly38.fasta, refs/Homo_sapiens_assembly38.fasta.fai

**Curated mutation data for HCC1143 from the DepMap portal:**

https://depmap.org/portal/cell_line/ACH-000374?tab=mutations

Files used: HCC1143 mutations.csv

All data are aligned to the GRCh38 / hg38 reference genome. 

## Objective

The objective of this notebook is to perform somatic variant calling on the HCC1143 tumor–normal WES dataset. 

## Workflow

The analysis follows the GATK Best Practices somatic variant calling workflow:

**BAM files → Mutect2 → Variant filtering → Functional annotation (MAF) → Comparison with known HCC1143 mutations**

All variant calling steps were performed using GATK v4.6.2.0 executed via the official Broad Institute Docker image.

## Somatic variant calling with Mutect2

The following command runs Mutect2 inside a GATK Docker container, using matched tumor and normal samples. 

Variant calling was restricted to chromosome 17. 

In addition to generating a somatic VCF, a BAM file containing Mutect2 realigned reads is produced for visualization and quality assessment in IGV.

A population allele-frequency resource derived from gnomAD was used to reduce false-positive calls from germline variants. The parameters --max-population-af and --af-of-alleles-not-in-resource define conservative population-frequency priors in Mutect2 and are consistent with GATK Best Practices for paired tumor–normal somatic variant calling.

In [ ]:
! gatk Mutect2 \
    -R /data/ref/Homo_sapiens_assembly38.fasta \
    -I /data/bams/tumor.bam \
    -I /data/bams/normal.bam \
    -tumor HCC1143_tumor \
    -normal HCC1143_normal \
    --germline-resource af-only-gnomad.hg38.vcf.gz \
    --af-of-alleles-not-in-resource 0.01 \
    --max-population-af 0.01 \
    -L chr17 \
    -O /data/somatic.chr17.vcf.gz \
    -bamout /data/mutect2_tumor_normal.chr17.bam

GnomAD filtering reduces false-positive somatic calls from common germline variants but does not account for cross-sample contamination. Therefore, the next step is contamination estimation.

## Сontamination estimation

Sample contamination was estimated using GetPileupSummaries and CalculateContamination based on common germline variants in the tumor and matched normal sample to improve specificity of downstream filtering.

GetPileupSummaries is performed independently for the tumor and the matched normal samples, based on a curated set of common germline SNPs (file small_exac_common_3.hg38.vcf.gz). GetPileupSummaries produces per-sample allele count summary tables. 

Then CalculateContamination compares observed allele fractions in the tumor to those expected based on the normal sample, identifies systematic VAF shifts at common germline SNPs and infers the most likely contamination fraction that explains these shifts. 

In [ ]:
!gatk GetPileupSummaries \
    -I bams/tumor.bam \
    -V small_exac_common_3.hg38.vcf.gz \
    -L small_exac_common_3.hg38.vcf.gz \
    -O tumor_summary.table

In [ ]:
!gatk GetPileupSummaries \
    -I bams/normal.bam \
    -V somatic-hg38-small_exac_common_3.hg38.vcf.gz \
    -L somatic-hg38-small_exac_common_3.hg38.vcf.gz \
    -O normal_summary.table

In [ ]:
!gatk CalculateContamination \
    -I tumor_summary.table \
    -matched normal_summary.table \
    -O contamination.table 

Inspecting the contamination.table, we observe an estimated contamination level of approximately ~1.1%, indicating low admixture in the tumor sample.

In [64]:
!cat contamination.table

sample	contamination	error
HCC1143_tumor	0.011485364960150258	0.0021979540554002835


##  Somatic variant filtering with FilterMutectCalls

In the next step, candidate somatic variants produced by Mutect2 assigns filter statuses using FilterMutectCalls (e.g. PASS, contamination,  technical artifact, low evidence and other flags). FilterMutectCalls applies a probabilistic model to integrate multiple sources of evidence, including sequencing depth, base and mapping quality, contamination, strand bias, and local haplotype context. 

The resulting VCF retains all candidate variants but provides explicit annotations that facilitate informed downstream selection of high-confidence somatic calls, rather than producing a pre-filtered variant set.

In [ ]:
!gatk FilterMutectCalls \
    -V somatic.chr17.vcf.gz \
    -R /data/ref/Homo_sapiens_assembly38.fasta \
    --contamination-table contamination.table \
    --stats somatic.chr17.vcf.gz.stats \
    -O somatic_postfilter.vcf.gz

## Functional annotation with Funcotator (VCF → MAF conversion)

Prior to annotation, the required Funcotator data sources for somatic variant analysis on the hg38 reference genome were downloaded using FuncotatorDataSourceDownloader. These data sources include gene models, transcript annotations, and multiple functional annotation tracks used to interpret variant consequences from COSMIC, ClinVar, Uniprot.

Functional annotation was then performed with Funcotator on the subset of somatic variants that passed FilterMutectCalls (flag "PASS"). Annotated variants were exported in both VCF and MAF formats - the VCF output preserves the complete technical variant representation and the MAF output provides a user-friendly tabular summary optimized for downstream interpretation and comparison with external datasets. 


In [ ]:
!gatk FuncotatorDataSourceDownloader \
  --somatic \
  --hg38 \
  --output funcotator_dataSources \
  --extract-after-download true

In [ ]:
!bcftools view -f PASS somatic_postfilter.vcf.gz -Oz -o somatic_PASS.vcf.gz

In [ ]:
!bcftools index somatic_PASS.vcf.gz

In [ ]:
!gatk Funcotator \
  --data-sources-path funcotator_dataSources.v1.8.hg38.20230908s \
  --ref-version hg38 \
  -R ref/Homo_sapiens_assembly38.fasta \
  -V somatic_PASS.vcf.gz \
  -O somatic_annot.vcf.gz \
  --output-file-format VCF

In [ ]:
!gatk Funcotator \
  --data-sources-path funcotator_dataSources.v1.8.hg38.20230908s \
  --ref-version hg38 \
  -R ref/Homo_sapiens_assembly38.fasta \
  -V somatic_PASSed.vcf.gz \
  -O somatic_annot.maf \
  --output-file-format MAF

## Сomparison with reference data for HCC1143

The annotated MAF file was loaded into a pandas DataFrame to enable downstream filtering and analysis. Relevant annotation fields were selected, including gene symbol, genomic coordinates, variant classification and type, alleles, codon and protein changes, functional annotations, and selected quality metrics (Tumor LOD). We focus on missense mutations in chromosome 17 gene-coding regions.

In [ ]:
import pandas as pd

In [25]:
muts_raw = pd.read_csv('somatic_annot.maf',sep="\t",comment="#")

In [27]:
col_selected=['Hugo_Symbol','Chromosome', 'Start_Position','End_Position', 'Variant_Classification', 'Variant_Type',
      'Reference_Allele', 'Tumor_Seq_Allele2','Tumor_Seq_Allele1', 'Codon_Change','TLOD', 
      'Protein_Change', 'GO_Molecular_Function', 'COSMIC_tissue_types_affected']

In [28]:
muts_selected=muts_raw[col_selected]

In [29]:
muts_selected = muts_selected[muts_selected['Variant_Classification'] == 'Missense_Mutation']
muts_selected = muts_selected.reset_index(drop=True)

In [30]:
muts_selected

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,Tumor_Seq_Allele1,Codon_Change,TLOD,Protein_Change,GO_Molecular_Function,COSMIC_tissue_types_affected
0,NLRP1,chr17,5541887,5541887,Missense_Mutation,SNP,C,T,C,c.(2668-2670)aGa>aAa,209.99,p.R890K,ATP binding (GO:0005524)|cysteine-type endopep...,biliary_tract(2)|breast(32)|central_nervous_sy...
1,FBXO39,chr17,6779878,6779878,Missense_Mutation,SNP,G,A,G,c.(10-12)Gaa>Aaa,37.12,p.E4K,NaN,biliary_tract(2)|breast(11)|central_nervous_sy...
2,TP53,chr17,7674220,7674220,Missense_Mutation,SNP,C,T,C,c.(742-744)cGg>cAg,264.57,p.R248Q,ATP binding (GO:0005524)|chaperone binding (GO...,NS(1495)|adrenal_gland(724)|autonomic_ganglia(...
3,ANKRD13B,chr17,29609163,29609163,Missense_Mutation,SNP,G,C,G,c.(643-645)Gac>Cac,22.18,p.D215H,NaN,biliary_tract(2)|breast(11)|central_nervous_sy...
4,LIG3,chr17,34983021,34983021,Missense_Mutation,SNP,A,G,A,c.(16-18)Aag>Gag,14.24,p.K6E,ATP binding (GO:0005524)|DNA binding (GO:00036...,biliary_tract(2)|breast(223)|central_nervous_s...
5,ABCA9,chr17,69016308,69016308,Missense_Mutation,SNP,A,G,A,c.(2983-2985)cTt>cCt,49.96,p.L995P,ATP binding (GO:0005524)|ATPase activity (GO:0...,biliary_tract(2)|breast(11)|central_nervous_sy...
6,GGA3,chr17,75239434,75239434,Missense_Mutation,SNP,G,A,G,c.(1504-1506)cCg>cTg,178.91,p.P502L,ADP-ribosylation factor binding (GO:0030306),biliary_tract(2)|breast(49)|central_nervous_sy...
7,CCDC137,chr17,81671752,81671752,Missense_Mutation,SNP,A,T,A,c.(505-507)aAg>aTg,86.57,p.K169M,poly(A) RNA binding (GO:0044822),biliary_tract(2)|central_nervous_system(45)|pa...
8,UTS2R,chr17,82374762,82374762,Missense_Mutation,SNP,C,G,C,c.(436-438)agC>agG,112.01,p.S146R,G-protein coupled receptor activity (GO:000493...,biliary_tract(2)|breast(48)|central_nervous_sy...


In [35]:
final_muts = muts_selected[['Hugo_Symbol','Start_Position']]
our_muts_results = final_muts.rename(columns={'Start_Position': 'Position'})

In [36]:
our_muts_results

,Hugo_Symbol,Position
0,NLRP1,5541887
1,FBXO39,6779878
2,TP53,7674220
3,ANKRD13B,29609163
4,LIG3,34983021
5,ABCA9,69016308
6,GGA3,75239434
7,CCDC137,81671752
8,UTS2R,82374762


To assess the biological plausibility of the detected variants, the filtered mutation set was compared against an external curated mutation dataset for the HCC1143 cell line. Reference mutation data were restricted to chromosome 17.

In [42]:
real_muts = pd.read_csv("HCC1143 mutations.csv")
real_muts=real_muts[real_muts['Chromosome']== 'chr17']
real_muts

,Gene,Chromosome,Position,Variant Type,Variant Info,Ref Allele,Alt Allele,Allele Fraction,Ref Count,Alt Count,...,NMD,Vep Somatic,Vep Impact,Oncogene High Impact,AM class,AM Pathogenicity,Hotspot,Intron,Exon,Rescue Reason
70,NLRP1,chr17,5541887,SNV,missense_variant,C,T,0.982,0,53,...,NaN,0&1,MODERATE,False,likely_benign,0.1477,False,NaN,6/17,NaN
71,FBXO39,chr17,6779878,SNV,missense_variant,G,A,0.384,32,20,...,NaN,1,MODERATE,False,likely_benign,0.1088,False,NaN,2/4,NaN
72,TP53,chr17,7674220,SNV,missense_variant,C,T,0.982,0,55,...,NaN,0&1&1&1&1&1,MODERATE,False,likely_pathogenic,0.9963,True,NaN,7/11,"OncoKB, Cosmic, Hess"
73,ANKRD13B,chr17,29609163,SNV,missense_variant,G,C,0.431,29,22,...,NaN,NaN,MODERATE,False,likely_benign,0.3163,False,NaN,6/15,NaN
74,LIG3,chr17,34983021,SNV,missense_variant,A,G,0.204,43,10,...,NaN,NaN,MODERATE,False,likely_benign,0.1235,False,NaN,2/20,NaN
75,SRSF1,chr17,58006457,SNV,missense_variant,G,A,0.170,71,14,...,NaN,1,MODERATE,False,likely_pathogenic,0.9649,False,NaN,2/4,NaN
76,ABCA9,chr17,69016308,SNV,missense_variant,A,G,0.220,66,18,...,NaN,NaN,MODERATE,False,likely_pathogenic,0.8596,False,NaN,22/39,NaN
77,GGA3,chr17,75239434,SNV,missense_variant,G,A,0.989,0,91,...,NaN,0&1,MODERATE,False,likely_benign,0.0538,False,NaN,14/17,NaN
78,CCDC137,chr17,81671752,SNV,missense_variant,A,T,0.402,55,38,...,NaN,NaN,MODERATE,False,likely_benign,0.1964,False,NaN,4/6,NaN
79,UTS2R,chr17,82374762,SNV,missense_variant,C,G,0.990,0,103,...,NaN,0&1&1,MODERATE,False,ambiguous,0.5180,False,NaN,3/3,NaN


In [46]:
real_muts_for_comparison= real_muts[['Gene','Position']]
real_muts_for_comparison = real_muts_for_comparison.rename(columns={'Gene': 'Hugo_Symbol'})
real_muts_for_comparison=real_muts_for_comparison.reset_index(drop=True)

In [47]:
real_muts_for_comparison

,Hugo_Symbol,Position
0,NLRP1,5541887
1,FBXO39,6779878
2,TP53,7674220
3,ANKRD13B,29609163
4,LIG3,34983021
5,SRSF1,58006457
6,ABCA9,69016308
7,GGA3,75239434
8,CCDC137,81671752
9,UTS2R,82374762


In [61]:
our_muts_results

,Hugo_Symbol,Position
0,NLRP1,5541887
1,FBXO39,6779878
2,TP53,7674220
3,ANKRD13B,29609163
4,LIG3,34983021
5,ABCA9,69016308
6,GGA3,75239434
7,CCDC137,81671752
8,UTS2R,82374762


## Conclusions and limitations

Somatic variant calling on chromosome 17 identified **9/10** missense SNVs reported for the HCC1143 cell line in the DepMap reference dataset. All recovered variants matched the reference data exactly at genomic position level, demonstrating high concordance between the applied GATK-based workflow and curated mutation profiles for this cell line.

Only one reference mutation, located in SRSF1, was not detected in the present analysis. The absence of this variant may be explained by multiple factors, including insufficient sequencing depth, low VAF due to subclonality or differences in cell line passage. Additionally, SRSF1 is located in a GC-rich genomic region, which can be challenging during library preparation and exome capture, leading to reduced amplification efficiency and decreased sensitivity for mutation detection at this locus. Notably, the missing SRSF1 mutation exhibits the lowest reported VAF in the reference real_muts dataset. 

## Possible improvements and future refinements

Further sensitivity could potentially be achieved by incorporating a Panel of Normals (PoN) to improve removal of recurrent technical artifacts. While PoN usage may not directly rescue true low-coverage variants, it can reduce background noise and increase confidence in borderline calls. 